### Tool Calling

In [1]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from dotenv import load_dotenv
load_dotenv()


True

### Tool Creation

In [2]:
@tool
def multiply(a: int , b: int) -> int:
    """Given 2 numbers a and b, this tool returns their product"""
    return a * b


In [3]:
print(multiply.invoke({'a': 3, 'b': 8}))

24


In [4]:
multiply.name

'multiply'

In [5]:
multiply.description

'Given 2 numbers a and b, this tool returns their product'

In [6]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

### Tool Binding

In [7]:
model = ChatGroq(model='llama-3.1-8b-instant')

In [8]:
model.invoke('Hi')

AIMessage(content="It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 36, 'total_tokens': 59, 'completion_time': 0.024107141, 'prompt_time': 0.041124059, 'queue_time': 0.045924701, 'total_time': 0.0652312}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--8407fa92-86bd-4804-8fb5-bc9741e847e5-0', usage_metadata={'input_tokens': 36, 'output_tokens': 23, 'total_tokens': 59})

In [9]:
llm_with_tools = model.bind_tools([multiply])

In [10]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="I'm functioning properly, thanks for asking. Is there anything I can help you with?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 227, 'total_tokens': 246, 'completion_time': 0.032573618, 'prompt_time': 0.022893977, 'queue_time': 0.049038799, 'total_time': 0.055467595}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--10d4171c-a6e0-4d22-a5a7-73e53ac25c34-0', usage_metadata={'input_tokens': 227, 'output_tokens': 19, 'total_tokens': 246})

In [11]:
query = HumanMessage('can you multiply 3 with 1000')

In [12]:
messages = [query]

In [13]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [14]:
result = llm_with_tools.invoke(messages)

In [15]:
messages.append(result)

In [16]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'he5mkn51t', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 232, 'total_tokens': 252, 'completion_time': 0.029674808, 'prompt_time': 0.745317369, 'queue_time': 0.36970475, 'total_time': 0.774992177}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ca9d93d1-a81e-4bb2-821f-1a6324125753-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'he5mkn51t', 'type': 'tool_call'}], usage_metadata={'input_tokens': 232, 'output_tokens': 20, 'total_tokens': 252})]

In [17]:
tool_result = multiply.invoke(result.tool_calls[0])

In [18]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='he5mkn51t')

In [19]:
messages.append(tool_result)

In [20]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'he5mkn51t', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 232, 'total_tokens': 252, 'completion_time': 0.029674808, 'prompt_time': 0.745317369, 'queue_time': 0.36970475, 'total_time': 0.774992177}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ca9d93d1-a81e-4bb2-821f-1a6324125753-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'he5mkn51t', 'type': 'tool_call'}], usage_metadata={'input_tokens': 232, 'output_tokens': 20, 'total_tokens': 252}),
 ToolMessage(content='3000', name='multiply', tool_call_id='he5mkn51t')]

In [22]:
llm_with_tools.invoke(messages).content

'The result of the multiplication is 3000.'